In [ ]:
# import config file which is in the same folder
import config

In [ ]:
dir(config)

### 1) Activating the Spotify API

In [ ]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))



# The "sp" variable has TWO USEFUl FUNCTIONS:
# The FIRST usefull function is:
# .search(q='',limit=n)
# .search(q="track:"+song_name+" artist:"+artist_name,limit=5) to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The SECOND usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.


In [ ]:
# Yann Tiersen is a composer
results = sp.search(q='Yann Tiersen', limit=5, market="GB")
results
#json_results = json.dumps(results, ensure_ascii=True)
#json_results
#results['tracks']['items'][0]['album']

In [ ]:
# in this dic the main key is "tracks"
results.keys()

In [ ]:
# the main keys of the main key "tracks" are the following (nested dictionary)
results['tracks'].keys()

In [ ]:
# prettyfing the printing

import pprint

pprint.pprint(results)

In [ ]:
len(results['tracks']['items'])

In [ ]:
results['tracks']['items'][0]

In [ ]:
results['tracks']['items'][0].keys()

In [ ]:
results['tracks']['items'][0]['id']

In [ ]:
# this code allows us to filter from any results JSON the track and the ID

for item in results['tracks']['items']:
    print("The name of song is: '{}' and the id is: {}".format(item['name'],item['external_urls']['spotify']))

In [ ]:
# identfying the features

import pandas as pd

song = sp.search(q="comptine d'un autre été", limit=1) 

#pprint.pprint(song['tracks']['items'][0]['uri']) -> we get the id
song["tracks"]["items"][0]["uri"]

#isolating the audio feautres of the track
sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

#creating a dictionary with the audio feature
my_dict = sp.audio_features(song["tracks"]["items"][0]["uri"])[0] 


# with this code you can potentally loop a lot of URIs and create a DF with
# the relevant feature

print(my_dict)
my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
#my_dict_new['name'] = [song["tracks"]["items"][0]['name']]
print(my_dict_new)


In [ ]:
# Put the result above into a DF
pd.DataFrame(my_dict_new)

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))


In [ ]:
result = sp.search("AJR")
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

In [ ]:
album = sp.album(track["album"]["external_urls"]["spotify"])
print("album genres:", album["genres"])
print("album release-date:", album["release_date"])

In [ ]:
# let s search for songs of genre = electronics

my_genre = ['edm_dance']

long_dict = {}
#for artist in my_artists:
#    #result = sp.search(q=artist, limit= 50)
#    my_dict[artist] = sp.search(q=artist, limit= 50)

long_dict = {genre: sp.search(q=genre, limit= 10000) for genre in my_genre}
#my_dict
long_dict.keys()

### 3) Loading tracks with features of a long playlist

In [1]:
import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
# import config file which is in the same folder
import config

### Defining a function for analyzing playlists¶


In [ ]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id='id',
    client_secret='secret'))

In [2]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentias
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))



# The "sp" variable has TWO USEFUl FUNCTIONS:
# The FIRST usefull function is:
# .search(q='',limit=n)
# .search(q="track:"+song_name+" artist:"+artist_name,limit=5) to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The SECOND usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.


In [9]:
def playlist_analyzer(user, playlist_id): 
    names = []
    uris = []
    artists = []
    popularity = []
    danceability = []
    energy = []
    key = []
    loudness = []
    mode = []
    speechiness = []
    acousticness = []
    instrumentalness =[]
    liveness=[]
    valence = []
    tempo =[]
    duration_ms =[]
    time_signature =[]
    playlist = spotify.user_playlist_tracks(user=user, playlist_id=playlist_id)
    tracks = playlist['items']
    
    while playlist['next']: # While statement to switch to next page of the playlist
        playlist = spotify.next(playlist)
        tracks.extend(playlist['items'])
   
    for item in tracks:
        names.append(item["track"]["name"])
        uri = item["track"]["uri"]
        uris.append(uri)
        popularity.append(item["track"]["popularity"])
        artists.append([artist["name"] for artist in item["track"]["artists"]])
        features = spotify.audio_features(uri)[0]
        danceability.append(features["danceability"])
        energy.append(features["energy"])
        key.append(features["key"])
        loudness.append(features["loudness"])
        mode.append(features["mode"])
        speechiness.append(features["speechiness"])
        acousticness.append(features["acousticness"])
        instrumentalness.append(features["instrumentalness"])
        liveness.append(features["liveness"])
        valence.append(features["valence"])
        tempo.append(features["tempo"])
        duration_ms.append(features["duration_ms"])
        time_signature.append(features["time_signature"])
    
    # Creating a df from with the song information
    df = pd.DataFrame({"names":names, 
                       "uri": uris,
                       "artists": artists,
                       "popularity": popularity,
                       "danceability":danceability,
                       "energy":energy,
                       "key": key,
                       "loudness":loudness,
                       "mode": mode,
                       "speechiness":speechiness,
                       "acousticness":acousticness,
                       "instrumentalness":instrumentalness,
                       "liveness": liveness,
                       "valence":valence,
                       "tempo": tempo,
                       "duration_ms":duration_ms,
                       "time_signature": time_signature})

    return df

### Getting data from Spotify playlist¶

In [11]:
playlist_df = playlist_analyzer("spotify", "spotify:playlist:6TY1by3xd2yDmwtZuKPKg1")
features_playlist = playlist_df.iloc[:,4:17].copy() #Creating a subset of the df with only the features

In [12]:
print(playlist_df)

                          names                                   uri  \
0                    Bad Habits  spotify:track:6PQ88X9TkUIAUIZJHW2upE   
1     STAY (with Justin Bieber)  spotify:track:5HCyWlXZPP0y6Gqq8TgA20   
2             You - Topic Remix  spotify:track:5bbtc9t6FcerXp732H2XNu   
3                    Heat Waves  spotify:track:02MWAaffLxlfxAUY7c5dvx   
4                      good 4 u  spotify:track:6PERP62TejQjgHu81OHxgM   
..                          ...                                   ...   
145               Hawái - Remix  spotify:track:0GzuHFG4Ql6DoyxFRnIk3F   
146  Tick Tock (feat. 24kGoldn)  spotify:track:27u7t9d7ZQoyjsCROHuZJ3   
147                    Lemonade  spotify:track:02kDW379Yfd5PzW5A6vuGt   
148       Roses - Imanbek Remix  spotify:track:24Yi9hE78yPEbZ4kxyoXAI   
149               Lose Somebody  spotify:track:1sgDyuLooyvEML4oHspNza   

                                       artists  popularity  danceability  \
0                                 [Ed Sheeran] 

In [13]:
print(features_playlist)

     danceability  energy  key  loudness  mode  speechiness  acousticness  \
0           0.808   0.897   11    -3.712     0       0.0348        0.0469   
1           0.591   0.764    1    -5.484     1       0.0483        0.0383   
2           0.733   0.887    6    -5.601     0       0.0692        0.0554   
3           0.761   0.525   11    -6.900     1       0.0944        0.4400   
4           0.556   0.661    6    -5.052     0       0.2040        0.3000   
..            ...     ...  ...       ...   ...          ...           ...   
145         0.766   0.788    6    -3.128     0       0.3120        0.1050   
146         0.779   0.705    0    -3.895     1       0.0344        0.3690   
147         0.799   0.660    1    -6.153     0       0.0790        0.2560   
148         0.770   0.724    8    -5.484     1       0.0495        0.0167   
149         0.580   0.586    1    -6.883     1       0.0357        0.3440   

     instrumentalness  liveness  valence    tempo  duration_ms  time_signat